# 1. Title, Name and References

## MLMaverick Team

#### Seulah Lee 300361747
#### David Steven Salazar Villegas 300356922
#### Ishwinder Singh 300351938

### 2.  Importing pandas libraries and csv file for further operations

In [1]:
import pandas as pd
df = pd.read_csv('bank-sampled-9k.csv',index_col=0)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
11,29,admin.,single,secondary,no,390,yes,no,unknown,5,may,137,1,-1,0,unknown,no
12,53,technician,married,secondary,no,6,yes,no,unknown,5,may,517,1,-1,0,unknown,no
17,57,blue-collar,married,primary,no,52,yes,no,unknown,5,may,38,1,-1,0,unknown,no


### PreProcessing and Removing outliers

In [2]:
condition = df.poutcome == 'other'
df_cleaned = df.drop(df[condition].index, axis = 0, inplace = False)

df_cleaned[['job','education']] = df_cleaned[['job','education']].replace(['unknown'],'other')
print(df_cleaned.shape)


from scipy.stats import zscore

df_cleaned[['balance']].mean()
df_cleaned[['balance']].mean()

df_cleaned['balance_outliers'] = df_cleaned['balance']
df_cleaned['balance_outliers']= zscore(df_cleaned['balance_outliers'])

condition1 = (df_cleaned['balance_outliers']>3) | (df_cleaned['balance_outliers']<-3 )
df_cleaned = df_cleaned.drop(df_cleaned[condition1].index, axis = 0, inplace = False)
df_cleaned = df_cleaned.drop('balance_outliers', axis=1)
print(df_cleaned.shape)

(8647, 17)
(8524, 17)


### 3. Dropping duration column and merging data of `date` and `day` columns into `last_contact`

In [3]:
df_cleaned.drop('duration',axis=1,inplace=True)

from datetime import datetime as dt

last_date = "18 Nov, 2010"
dt_1 = dt.strptime(last_date, "%d %b, %Y")

df_cleaned['month'] = df_cleaned['month'].apply(lambda x: x.capitalize())
df_cleaned['date'] = df_cleaned['day'].astype(str) + " " + df_cleaned['month'] + ", 2010"

df_cleaned['last_contact'] = df_cleaned['date'].apply(lambda x: (dt_1 - dt.strptime(x, "%d %b, %Y")).days)

df_cleaned.drop(columns=['day', 'date', 'month'], axis=1, inplace=True)

df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
11,29,admin.,single,secondary,no,390,yes,no,unknown,5,may,137,1,-1,0,unknown,no
12,53,technician,married,secondary,no,6,yes,no,unknown,5,may,517,1,-1,0,unknown,no
17,57,blue-collar,married,primary,no,52,yes,no,unknown,5,may,38,1,-1,0,unknown,no
18,60,retired,married,primary,no,60,yes,no,unknown,5,may,219,1,-1,0,unknown,no
24,40,retired,married,primary,no,0,yes,yes,unknown,5,may,181,1,-1,0,unknown,no
38,36,admin.,divorced,secondary,no,506,yes,no,unknown,5,may,577,1,-1,0,unknown,no
40,44,services,divorced,secondary,no,2586,yes,no,unknown,5,may,160,1,-1,0,unknown,no
50,54,blue-collar,married,secondary,no,1291,yes,no,unknown,5,may,266,1,-1,0,unknown,no


### Display the summary statistics of the dataset

In [4]:
df_cleaned.describe()

,age,balance,campaign,pdays,previous,last_contact
count,8524.000000,8524.000000,8524.000000,8524.000000,8524.000000,8524.000000
mean,41.045753,1080.669052,2.786954,32.301267,0.436415,149.692985
std,10.523894,1715.537388,3.061747,90.824835,1.630640,73.372969
min,18.000000,-4057.000000,1.000000,-1.000000,0.000000,-42.000000
25%,33.000000,63.000000,1.000000,-1.000000,0.000000,105.000000
50%,39.000000,430.000000,2.000000,-1.000000,0.000000,166.000000
75%,49.000000,1363.250000,3.000000,-1.000000,0.000000,191.000000
max,90.000000,11115.000000,51.000000,792.000000,55.000000,316.000000


In [5]:
df_cleaned.dtypes

age              int64
job             object
marital         object
education       object
default         object
balance          int64
housing         object
loan            object
contact         object
campaign         int64
pdays            int64
previous         int64
poutcome        object
y               object
last_contact     int64
dtype: object

### Dummy variable creation for categorical columns

- Used a new variable prefix to store the column name and used a for loop to iterate over all the object datatypes
- After that appended each of dummy variable to the prefix. used a valid name for each column 
- changed any appearance of hypen ‘-‘ with underscore ‘_’

In [6]:
categorical_columns = df_cleaned.select_dtypes('object').columns
dummy_dfs = []

for column in categorical_columns:
    prefix = column
    dummies = pd.get_dummies(df_cleaned[column], prefix=prefix, drop_first=True)
    dummies.columns = dummies.columns.map(lambda x: x.replace('-', '_'))
    dummies = dummies.astype(int)
    dummy_dfs.append(dummies)
    df_cleaned.drop(column, axis=1, inplace=True)

df_cleaned = pd.concat([df_cleaned] + dummy_dfs, axis=1)

print(df_cleaned)

       age  balance  campaign  pdays  previous  last_contact  job_blue_collar  \
0       58     2143         1     -1         0           197                0   
2       33        2         1     -1         0           197                0   
11      29      390         1     -1         0           197                0   
12      53        6         1     -1         0           197                0   
17      57       52         1     -1         0           197                1   
...    ...      ...       ...    ...       ...           ...              ...   
45196   25      358         1     -1         0             2                0   
45201   53      583         1    184         4             1                0   
45204   73     2850         1     40         8             1                0   
45206   51      825         3     -1         0             1                0   
45207   71     1729         2     -1         0             1                0   

       job_entrepreneur  jo

# 4. Feature Selection and Scaling
- Logistic Regression with max interation to use estimator for threshold

In [7]:

## Dropping the target variable to make a features array
X = df_cleaned.drop('y_yes', axis=1)
y = df_cleaned['y_yes']
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [8]:
method_logreg = SelectFromModel(estimator=LogisticRegression(max_iter=5000)) 
method_logreg.fit(X, y)

pd.DataFrame({'coef': method_logreg.estimator_.coef_[0], 
            'select_feature' : method_logreg.get_support()}, 
             index=X.columns)

,coef,select_feature
age,-0.005312,False
balance,0.000081,False
campaign,-0.121232,False
pdays,-0.001980,False
previous,0.028989,False
last_contact,0.001132,False
job_blue_collar,0.006223,False
job_entrepreneur,-0.212416,False
job_housemaid,-0.303468,True
job_management,-0.221429,False


### Scaling features and then checking shape of feature array

In [9]:
method_logreg.threshold_
method_logreg.get_support()
X_2 = method_logreg.transform(X)
X_2.shape

## X_2 is the feature array selected with 7 features.

(8524, 8)

### Feature Selection using Support Vector Machines  hyperparameter value 0.001

In [10]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel


svm = LinearSVC(C=0.001, penalty='l1', dual=False)
svm.fit(X, y)


model_svm = SelectFromModel(svm, prefit=True)
selected_features_svm = model_svm.get_support(indices=True)

print("Selected Features using Linear SVM:")
print(selected_features_svm)


## selected_features_svm is the feature selection array for svm

Selected Features using Linear SVM:
[ 0  1  2  3  5 23]


### Feature Selection using Select K Best  

In [11]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=10)
X_selected = selector.fit_transform(X, y)


selected_feature_k = selector.get_support(indices=True)


selected_columns_k = X.columns[selected_feature_k];
print("Selected Features using SelectKBest with mutual_info_classif:")
print(selected_feature_k)


## selected_columns_k is the feature selection array for selectk best 

Selected Features using SelectKBest with mutual_info_classif:
[ 0  3  4  5 18 22 23 26 27 28]


### Decision tree Classifier

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features_svm, y, test_size=0.25, random_state=42)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

train_acc = tree.score(X_train, y_train)
test_acc = tree.score(X_test, y_test)

print('Training Accuracy:', train_acc)
print('Test Accuracy:', test_acc)


ValueError: Found input variables with inconsistent numbers of samples: [6, 8524]